### How many max features?

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import mlflow

# Setup MLFlow Tracking Server
mlflow_tracking_uri = os.getenv("MLFLOW_TRACKING_URI")
mlflow.set_tracking_uri(mlflow_tracking_uri)

e:\Machine Learning\YouTube_Comment_Analyzer\.venv\Lib\site-packages\mlflow\protos\service_pb2.py:11: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service
e:\Machine Learning\YouTube_Comment_Analyzer\.venv\Lib\site-packages\mlflow\utils\requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


In [3]:
# Set or Create an experiment
mlflow.set_experiment("Expr 3- TFIDF Trigram max features")

2025/12/26 13:30:20 INFO mlflow.tracking.fluent: Experiment with name 'Expr 3- TFIDF Trigram max features' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/bf26a879d5224a2a8c45bf2ed05e7792', creation_time=1766736019531, experiment_id='4', last_update_time=1766736019531, lifecycle_stage='active', name='Expr 3- TFIDF Trigram max features', tags={}>

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [5]:
df = pd.read_csv('./artifacts/reddit_preprocessing.csv').dropna(subset=['clean_comment'])
print(df.shape)
df.sample(5)

(36662, 2)


,clean_comment,category
9740,bhug,0
15556,effortlessly karmawhoring,0
14633,let raise glass defend freedom paid ultimate p...,0
1735,modi watch south park,0
16144,charged folk india month subscribe billion peo...,0


In [12]:
import os

ARTIFACT_DIR = "artifacts"
os.makedirs(ARTIFACT_DIR, exist_ok=True)

In [13]:
# Function to run the experiment
def run_experiment_tfidf_max_features(max_features):
    ngram_range = (1, 3)    # Trigram setting

    # Vectorization using TF-IDF with varying max_features
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42)

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Define and train a Random Forest Model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"TFIDF_Trigram_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a descripion
        mlflow.set_tag("description", f"Random Forest with TF-IDF Trigrams, max_features={max_features}")

         # Log Vectorizer parameters
        mlflow.log_param("vectorizer_type: ", "TF-IDF")
        mlflow.log_param("ngram_range: ", ngram_range)
        mlflow.log_param("vectorizer_max_features: ", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Make predictions and log metrics
        y_pred = model.predict(X_test)

        # Log Accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("Accuracy: ", accuracy)

        # Log Classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    if metric != "support":
                        mlflow.log_metric(f"{label}_{metric}", value)

        # Log Confusion Matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, max_features={max_features}")

        # Save and log the confusion matrix plot
        filename = f"Exp3_TF_IDF_Trigrams{max_features}_ConfusionMatrix.png"
        cm_path = os.path.join(ARTIFACT_DIR, filename)
        plt.savefig(cm_path)
        mlflow.log_artifact(cm_path)
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"Random_Forest_Model_TF_IDF_Trigrams_{max_features}")


In [14]:
# Test various max_features values
max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_tfidf_max_features(max_features)

2025/12/26 13:58:17 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
e:\Machine Learning\YouTube_Comment_Analyzer\.venv\Lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
2025/12/26 13:59:00 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
e:\Machine Learning\YouTube_Comment_Analyzer\.venv\Lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register conc